In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/midiclassics.zip'
extract_path = '/content/'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Check folder structure
os.listdir(extract_path)

['.config',
 'Rothchild Piano Sonata Rmw13 1mov.mid',
 'Vaughan',
 'Reinecke Piano Concerto n3 3mov.mid',
 'Dvorak Symphony op70 n7 3mov.mid',
 'Lange',
 'Dvorak Symphony op70 n7 2mov.mid',
 'Rothchild Oboe Concerto Rmw09 3mov.mid',
 'Hemery',
 'Botsford',
 'Field',
 'Katzwarra',
 'Herold',
 'Debussy Suite Bergamasque 4mov.mid',
 'Stravinski',
 'Beethoven',
 'Gershuin Rhapsody In Blue Piano Duet.mid',
 'Alkan',
 'Gottschalk',
 'German',
 'Jensen',
 'Grainger',
 'Buxethude Buxwv158 Preambulum.mid',
 'Chabrier',
 'Handel',
 'Bach',
 'Shostakovich',
 'Busser',
 'Tchaikovsky Lake Of The Swans Act 2 14mov.mid',
 'Borodin',
 'Tchaikovsky Lake Of The Swans Act 1 6mov.mid',
 'Brahms',
 'Berlin',
 'Schubert',
 'Grieg',
 'Pachebel Toccata n2.mid',
 'Pollen Beguine Royale.mid',
 'Kuhlau Sonatina op60 n3.mid',
 'Holst',
 'Rothchild Oboe Concerto Rmw09 1mov.mid',
 'Peterson-Berger',
 'Rothchild Oboe Concerto Rmw09 2mov.mid',
 'Reinecke Piano Concerto n3 1mov.mid',
 'Holst, M',
 'Barber',
 'Lecuona'

In [9]:
import os
import glob
import warnings

warnings.filterwarnings("ignore")

def gather_distinct_midis(folder_path):
    midi_path_set = set()
    distinct_midi_files = []

    midi_file_list = glob.glob(os.path.join(folder_path, '**', '*.mid'), recursive=True)

    for midi in midi_file_list:
        full_path = os.path.abspath(midi)
        if full_path not in midi_path_set:
            midi_path_set.add(full_path)
            distinct_midi_files.append(full_path)

    return distinct_midi_files

In [10]:
# Bach directory
bach_folder_path = '/content/Bach'
bach_midi_list = gather_distinct_midis(bach_folder_path)
print(f"Number of MIDI files found for Bach: {len(bach_midi_list)}")

Number of MIDI files found for Bach: 925


In [11]:
# Beethoven directory
beethoven_folder_path = '/content/Beethoven'
beethoven_midi_list = gather_distinct_midis(beethoven_folder_path)
print(f"Number of MIDI files found for Beethoven: {len(beethoven_midi_list)}")

Number of MIDI files found for Beethoven: 212


In [12]:
# Chopin directory
chopin_folder_path = '/content/Chopin'
chopin_midi_list = gather_distinct_midis(chopin_folder_path)
print(f"Number of MIDI files found for Chopin: {len(chopin_midi_list)}")

Number of MIDI files found for Chopin: 136


In [13]:
# Mozart directory
mozart_folder_path = '/content/Mozart'
mozart_midi_list = gather_distinct_midis(mozart_folder_path)
print(f"Number of MIDI files found for Mozart: {len(mozart_midi_list)}")

Number of MIDI files found for Mozart: 257


In [14]:
from music21 import converter, note, chord, tempo, meter
import numpy as np
from fractions import Fraction

# Utility Functions

def to_float(value):
    if isinstance(value, Fraction):
        return float(value)
    return float(value)

def complete_chord_row(chord_row, max_pitches=4, fill_note=0):
    start_time = to_float(chord_row[0])
    chord_pitches = chord_row[1:]
    chord_pitches += [fill_note] * (max_pitches - len(chord_pitches))
    return [start_time] + chord_pitches

def pad_feature_table(feature_rows, expected_length, fill_note=0):
    result = []
    for row in feature_rows:
        row = row + [fill_note] * (expected_length - len(row))
        result.append(row)
    return np.array(result, dtype=float)

In [15]:
# Feature Extraction

def extract_score_data(parsed_score, max_chord_count=4):
    note_data            = []
    chord_data           = []
    tempo_data           = []
    rhythm_pattern_data  = []
    time_signature_data  = []

    for item in parsed_score.flat:
        if isinstance(item, note.Note):
            note_data.append([
                to_float(item.offset),
                item.pitch.midi,
                to_float(item.quarterLength),
                item.volume.realized
            ])
            rhythm_pattern_data.append([
                to_float(item.offset),
                to_float(item.quarterLength)
            ])
        elif isinstance(item, chord.Chord):
            this_chord = [to_float(item.offset)] + [p.midi for p in item.pitches]
            chord_data.append(complete_chord_row(this_chord, max_pitches=max_chord_count))

    for t in parsed_score.flat.getElementsByClass(tempo.MetronomeMark):
        tempo_data.append([to_float(t.offset), t.number])

    for sig in parsed_score.flat.getElementsByClass(meter.TimeSignature):
        time_signature_data.append([to_float(sig.offset), sig.numerator, sig.denominator])

    max_col_notes      = max((len(r) for r in note_data), default=0)
    max_col_chords     = max((len(r) for r in chord_data), default=0)
    max_col_tempo      = max((len(r) for r in tempo_data), default=0)
    max_col_rhythm     = max((len(r) for r in rhythm_pattern_data), default=0)
    max_col_timesig    = max((len(r) for r in time_signature_data), default=0)

    expected_cols = max(max_col_notes, max_col_chords, max_col_tempo, max_col_rhythm, max_col_timesig)

    note_array         = pad_feature_table(note_data, expected_cols)
    chord_array        = pad_feature_table(chord_data, expected_cols)
    tempo_array        = pad_feature_table(tempo_data, expected_cols)
    rhythm_array       = pad_feature_table(rhythm_pattern_data, expected_cols)
    timesig_array      = pad_feature_table(time_signature_data, expected_cols)

    min_row_count = min(
        len(note_array), len(chord_array), len(tempo_array),
        len(rhythm_array), len(timesig_array)
    )

    note_array     = note_array[:min_row_count]
    chord_array    = chord_array[:min_row_count]
    tempo_array    = tempo_array[:min_row_count]
    rhythm_array   = rhythm_array[:min_row_count]
    timesig_array  = timesig_array[:min_row_count]

    def force_2d(matrix):
        if matrix.ndim == 1:
            return matrix.reshape(-1, 1)
        return matrix

    note_array     = force_2d(note_array)
    chord_array    = force_2d(chord_array)
    tempo_array    = force_2d(tempo_array)
    rhythm_array   = force_2d(rhythm_array)
    timesig_array  = force_2d(timesig_array)

    combined_array = np.hstack([note_array, chord_array, tempo_array, rhythm_array, timesig_array]) \
        if min_row_count > 0 else np.empty((0, 0))
    return combined_array


In [16]:
# Collect features for composers

def collect_composer_data(composer_title, midi_path_list, max_files=136):
    all_composer_features = []
    composer_label_list = []
    midi_used_files = []

    i = 0
    for midi_path in midi_path_list:
        if i >= max_files:
            break
        try:
            music_score = converter.parse(midi_path)
            data_matrix = extract_score_data(music_score)
            all_composer_features.append(data_matrix)
            composer_label_list.append(composer_title)
            midi_used_files.append(midi_path)
            i += 1
        except Exception:
            continue

    return np.array(all_composer_features, dtype=object), np.array(composer_label_list), np.array(midi_used_files)


In [18]:

bach_feature_arrays, bach_label_list, bach_file_list = collect_composer_data("Bach", bach_midi_list)
print(f"Bach dataset: Feature arrays extracted from {len(bach_feature_arrays)} files.")

Bach dataset: Feature arrays extracted from 136 files.


In [19]:
beethoven_feature_arrays, beethoven_label_list, beethoven_file_list = collect_composer_data("Beethoven", beethoven_midi_list)
print(f"Beethoven dataset: Feature arrays extracted from {len(beethoven_feature_arrays)} files.")

Beethoven dataset: Feature arrays extracted from 136 files.


In [20]:
chopin_feature_arrays, chopin_label_list, chopin_file_list = collect_composer_data("Chopin", chopin_midi_list)
print(f"Chopin dataset: Feature arrays extracted from {len(chopin_feature_arrays)} files.")

Chopin dataset: Feature arrays extracted from 136 files.


In [21]:
mozart_feature_arrays, mozart_label_list, mozart_file_list = collect_composer_data("Mozart", mozart_midi_list)
print(f"Mozart dataset: Feature arrays extracted from {len(mozart_feature_arrays)} files.")

Mozart dataset: Feature arrays extracted from 136 files.


In [33]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")

# --- Padding function with truncation if too large ---
def pad_feature_cubes(feature_cubes, max_row_count, pad_entry=0.0):
    """
    Pad or truncate each 2D feature matrix in the 3D array collection to have shape
    (max_row_count, max_columns), ensuring consistent input shapes.
    """
    padded_cubes = []
    max_column_count = max(matrix.shape[1] for matrix in feature_cubes)
    for matrix in feature_cubes:
        truncated = matrix[:max_row_count, :]  # truncate if too many rows
        rows, cols = truncated.shape
        temp_padded = np.full((max_row_count, max_column_count), pad_entry, dtype=float)
        temp_padded[:rows, :cols] = truncated
        padded_cubes.append(temp_padded)
    return np.array(padded_cubes)

In [34]:
# --- Standardize (normalize) features ---
def standardize_feature_cubes(feature_cubes, max_row_count):
    padded_cubes = pad_feature_cubes(feature_cubes, max_row_count)
    flattened = padded_cubes.reshape(padded_cubes.shape[0], -1)
    scaler = StandardScaler()
    scaled = scaler.fit_transform(flattened)
    reshaped = scaled.reshape(padded_cubes.shape)
    return reshaped

In [35]:
# --- Pad to max width for uniform column count ---
def pad_to_max_width(feature_set, final_width, pad_fill=0.0):
    output = []
    for matrix in feature_set:
        rows, cols = matrix.shape
        filled = np.full((rows, final_width), pad_fill, dtype=float)
        filled[:, :cols] = matrix
        output.append(filled)
    return np.array(output)

In [36]:
# --- ==== MAIN PREPROCESSING ====

# Set desired fixed height and width matching model input_shape
DESIRED_HEIGHT = 168
DESIRED_WIDTH = 85

# Standardize features with fixed height
bach_features_standardized = standardize_feature_cubes(bach_feature_arrays, DESIRED_HEIGHT)
beethoven_features_standardized = standardize_feature_cubes(beethoven_feature_arrays, DESIRED_HEIGHT)
chopin_features_standardized = standardize_feature_cubes(chopin_feature_arrays, DESIRED_HEIGHT)
mozart_features_standardized = standardize_feature_cubes(mozart_feature_arrays, DESIRED_HEIGHT)

In [37]:
# Determine max width after standardization (should usually be consistent)
max_col_count = max(
    bach_features_standardized.shape[2],
    beethoven_features_standardized.shape[2],
    chopin_features_standardized.shape[2],
    mozart_features_standardized.shape[2]
)

In [38]:
# Pad to uniform max width
bach_features_final = pad_to_max_width(bach_features_standardized, max_col_count)
beethoven_features_final = pad_to_max_width(beethoven_features_standardized, max_col_count)
chopin_features_final = pad_to_max_width(chopin_features_standardized, max_col_count)
mozart_features_final = pad_to_max_width(mozart_features_standardized, max_col_count)

In [39]:
# Confirm shapes
print(f"Bach features shape:      {bach_features_final.shape}")
print(f"Beethoven features shape: {beethoven_features_final.shape}")
print(f"Chopin features shape:    {chopin_features_final.shape}")
print(f"Mozart features shape:    {mozart_features_final.shape}")

Bach features shape:      (136, 168, 85)
Beethoven features shape: (136, 168, 85)
Chopin features shape:    (136, 168, 85)
Mozart features shape:    (136, 168, 85)


In [40]:
# === Train/Test split per composer ===
bach_x_train, bach_x_test, bach_y_train, bach_y_test = train_test_split(
    bach_features_final, bach_label_list, test_size=0.2, random_state=42)
beethoven_x_train, beethoven_x_test, beethoven_y_train, beethoven_y_test = train_test_split(
    beethoven_features_final, beethoven_label_list, test_size=0.2, random_state=42)
chopin_x_train, chopin_x_test, chopin_y_train, chopin_y_test = train_test_split(
    chopin_features_final, chopin_label_list, test_size=0.2, random_state=42)
mozart_x_train, mozart_x_test, mozart_y_train, mozart_y_test = train_test_split(
    mozart_features_final, mozart_label_list, test_size=0.2, random_state=42)

In [41]:
# === Combine all for training and testing ===
x_train_all = np.concatenate([bach_x_train, beethoven_x_train, chopin_x_train, mozart_x_train], axis=0)
x_test_all = np.concatenate([bach_x_test, beethoven_x_test, chopin_x_test, mozart_x_test], axis=0)
y_train_all = np.concatenate([bach_y_train, beethoven_y_train, chopin_y_train, mozart_y_train], axis=0)
y_test_all = np.concatenate([bach_y_test, beethoven_y_test, chopin_y_test, mozart_y_test], axis=0)

print(f"Combined training features shape: {x_train_all.shape}")
print(f"Combined testing features shape: {x_test_all.shape}")
print(f"Combined training labels shape: {y_train_all.shape}")
print(f"Combined testing labels shape: {y_test_all.shape}")

Combined training features shape: (432, 168, 85)
Combined testing features shape: (112, 168, 85)
Combined training labels shape: (432,)
Combined testing labels shape: (112,)


In [42]:
# --- Encode labels numerically ---
composer_label_map = {"Bach":0, "Beethoven":1, "Chopin":2, "Mozart":3}
y_train_all_encoded = np.array([composer_label_map[label] for label in y_train_all])
y_test_all_encoded  = np.array([composer_label_map[label] for label in y_test_all])

print(f"Encoded training labels shape: {y_train_all_encoded.shape}")
print(f"Encoded testing labels shape: {y_test_all_encoded.shape}")

Encoded training labels shape: (432,)
Encoded testing labels shape: (112,)


In [43]:
# --- Add channel dimension for CNN ---
x_train_all = np.expand_dims(x_train_all, axis=-1)  # Shape now: (samples, 168, 85, 1)
x_test_all = np.expand_dims(x_test_all, axis=-1)

print(f"x_train_all shape after channel add: {x_train_all.shape}")
print(f"x_test_all shape after channel add: {x_test_all.shape}")

x_train_all shape after channel add: (432, 168, 85, 1)
x_test_all shape after channel add: (112, 168, 85, 1)


In [44]:
# === Define CNN model matching input shape ===
num_composer_classes = 4

composer_cnn_model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(DESIRED_HEIGHT, DESIRED_WIDTH, 1)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(128, (3,3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(num_composer_classes, activation='softmax')
])

In [45]:
composer_cnn_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [46]:
composer_cnn_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 166, 83, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 83, 41, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 81, 39, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 40, 19, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 38, 17, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 82688)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │     5,292,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,385,028 (20.54 MB)

 Trainable params: 5,385,028 (20.54 MB)

 Non-trainable params: 0 (0.00 B)

In [47]:
# --- Train the model ---
training_history = composer_cnn_model.fit(
    x_train_all, y_train_all_encoded,
    epochs=10, batch_size=32,
    validation_data=(x_test_all, y_test_all_encoded)
)

Epoch 1/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.5606 - loss: 0.9187 - val_accuracy: 1.0000 - val_loss: 0.0083
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 1.0000 - loss: 0.0051 - val_accuracy: 1.0000 - val_loss: 0.0012
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 1.0000 - loss: 6.5534e-04 - val_accuracy: 1.0000 - val_loss: 2.8111e-05
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 19s 956ms/step - accuracy: 1.0000 - loss: 3.4076e-05 - val_accuracy: 1.0000 - val_loss: 9.4460e-06
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 1.0000 - loss: 6.7272e-06 - val_accuracy: 1.0000 - val_loss: 5.2339e-06
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 1.0000 - loss: 1.9692e-06 - val_accuracy: 1.0000 - val_loss: 3.8303e-06
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 1.0000 - loss: 1.7520e-06 - val_accuracy: 1.0000 - val_loss: 3.2078e-06
Epoch 8/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 19s 952ms/step - accuracy: 1.0000 - l

In [48]:
# --- Evaluation ---
test_loss, test_accuracy = composer_cnn_model.evaluate(x_test_all, y_test_all_encoded)
print(f"Test accuracy: {test_accuracy}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 186ms/step - accuracy: 1.0000 - loss: 2.1789e-06
Test accuracy: 1.0


In [49]:
# --- Prediction & classification report ---
predictions = composer_cnn_model.predict(x_test_all)
predicted_classes = np.argmax(predictions, axis=1)

print("\n=== Expected labels ===")
print(y_test_all_encoded)
print("\n=== Predicted labels ===")
print(predicted_classes)

print("\n=== Classification Report ===")
print(classification_report(y_test_all_encoded, predicted_classes))

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 200ms/step

=== Expected labels ===
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3]

=== Predicted labels ===
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3]

=== Classification Report ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        28
           1       1.00      1.00      1.00        28
           2       1.00      1.00      1.00        28
           3       1.00      1.00      1.00        28

    accuracy                           1.00       112
   macro avg       1.00      1.00      1.00       112
weighted avg       1.00      1.00      1.

In [50]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import classification_report

num_composers = 4  # Number of output classes

# Define CNN architecture
composer_cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(168, 85, 1)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(num_composers, activation='softmax')  # Output layer with softmax activation
])


In [51]:
# Compile model with optimizer, loss, and metric
composer_cnn.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Print model architecture summary
composer_cnn.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 166, 83, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 83, 41, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 108896)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │     6,969,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,969,988 (26.59 MB)

 Trainable params: 6,969,988 (26.59 MB)

 Non-trainable params: 0 (0.00 B)

In [52]:
# Train the model
training_history = composer_cnn.fit(
    x_train_all, y_train_all_encoded,
    epochs=5,
    batch_size=10,
    validation_data=(x_test_all, y_test_all_encoded)
)

Epoch 1/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 8s 142ms/step - accuracy: 0.7982 - loss: 1.0529 - val_accuracy: 0.9821 - val_loss: 0.0530
Epoch 2/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 12s 194ms/step - accuracy: 0.9959 - loss: 0.0586 - val_accuracy: 0.9911 - val_loss: 0.0435
Epoch 3/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 12s 247ms/step - accuracy: 0.9915 - loss: 0.0287 - val_accuracy: 0.9911 - val_loss: 0.0570
Epoch 4/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 18s 191ms/step - accuracy: 0.9902 - loss: 0.0186 - val_accuracy: 0.9911 - val_loss: 0.0491
Epoch 5/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 9s 158ms/step - accuracy: 0.9969 - loss: 0.0091 - val_accuracy: 0.9911 - val_loss: 0.0460


In [53]:
# Evaluate on test data
test_loss, test_accuracy = composer_cnn.evaluate(x_test_all, y_test_all_encoded)
print(f"Test accuracy: {test_accuracy:.4f}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.9943 - loss: 0.0313
Test accuracy: 0.9911


In [54]:
# Generate predictions for test set
test_predictions = composer_cnn.predict(x_test_all)
predicted_labels = test_predictions.argmax(axis=1)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step


In [55]:
print("\n" + "="*40)
print("        Expected Labels")
print("="*40)
print(y_test_all_encoded)
print("\n")

print("="*40)
print("        Predicted Labels")
print("="*40)
print(predicted_labels)
print("\n")

# Print classification report
classification_rep = classification_report(y_test_all_encoded, predicted_labels)
print("="*40)
print("      CLASSIFICATION REPORT")
print("="*40)
print(classification_rep)


        Expected Labels
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3]


        Predicted Labels
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 3 3 3 0 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3]


      CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.97      1.00      0.98        28
           1       1.00      1.00      1.00        28
           2       1.00      1.00      1.00        28
           3       1.00      0.96      0.98        28

    accuracy                           0.99       112
   macro avg       0.99      0.99      0.99       112
weighted avg       0.99      0.99      0.99       112



In [56]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import classification_report

# Define input data dimensions
sequence_length = 168  # number of time steps per sample
feature_dim = 85       # features per time step
num_composers = 4      # output classes (e.g., 4 composers)

# Build LSTM model architecture
composer_lstm_model = Sequential([
    LSTM(128, input_shape=(sequence_length, feature_dim), return_sequences=True),
    Dropout(0.2),
    LSTM(128),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(num_composers, activation='softmax')
])

# Compile the model
composer_lstm_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Display model summary
composer_lstm_model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 168, 128)       │       109,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 168, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 249,668 (975.27 KB)

 Trainable params: 249,668 (975.27 KB)

 Non-trainable params: 0 (0.00 B)

In [57]:
# Train model for 20 epochs with batch size 32
training_history = composer_lstm_model.fit(
    x_train_all, y_train_all_encoded,
    epochs=20,
    batch_size=32,
    validation_data=(x_test_all, y_test_all_encoded)
)

Epoch 1/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 14s 662ms/step - accuracy: 0.4033 - loss: 1.2915 - val_accuracy: 0.5000 - val_loss: 0.9927
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 9s 676ms/step - accuracy: 0.4957 - loss: 0.9473 - val_accuracy: 0.5000 - val_loss: 0.8516
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 8s 495ms/step - accuracy: 0.4659 - loss: 0.8784 - val_accuracy: 0.6964 - val_loss: 0.8954
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 11s 550ms/step - accuracy: 0.5980 - loss: 0.8436 - val_accuracy: 0.7054 - val_loss: 0.7187
Epoch 5/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 9s 680ms/step - accuracy: 0.6356 - loss: 0.8295 - val_accuracy: 0.5000 - val_loss: 0.9380
Epoch 6/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 9s 563ms/step - accuracy: 0.4887 - loss: 0.9735 - val_accuracy: 0.5000 - val_loss: 0.9037
Epoch 7/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 10s 547ms/step - accuracy: 0.5407 - loss: 0.8670 - val_accuracy: 0.5000 - val_loss: 0.8429
Epoch 8/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 10s 704ms/step - accuracy: 0.4855 - loss: 0.8607 - val_accuracy

In [58]:
# Evaluate the model on test set
test_loss, test_accuracy = composer_lstm_model.evaluate(x_test_all, y_test_all_encoded)
print(f"Test accuracy: {test_accuracy:.4f}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 136ms/step - accuracy: 0.4542 - loss: 0.7970
Test accuracy: 0.5000


In [59]:
# Perform predictions on test set
test_predictions = composer_lstm_model.predict(x_test_all)
predicted_labels = np.argmax(test_predictions, axis=1)

3/4 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 233ms/step


In [60]:
print("\n" + "="*40)
print("        Expected Labels")
print("="*40)
print(y_test_all_encoded)
print("\n")

print("="*40)
print("        Predicted Labels")
print("="*40)
print(predicted_labels)
print("\n")

# Print classification report for in-depth metrics
classification_rep = classification_report(y_test_all_encoded, predicted_labels)
print("="*40)
print("      CLASSIFICATION REPORT")
print("="*40)
print(classification_rep)


        Expected Labels
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3]


        Predicted Labels
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2]


      CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        28
           1       1.00      1.00      1.00        28
           2       0.33      1.00      0.50        28
           3       0.00      0.00      0.00        28

    accuracy                           0.50       112
   macro avg       0.33      0.50      0.38       112
weighted avg       0.33      0.50      0.38       112



In [66]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import classification_report

# Define input dimensions
sequence_length = 168  # number of time steps per sample
feature_dimension = 85  # features per timestep
num_composer_classes = 4  # number of output classes (composers)

# Build the LSTM model architecture
composer_lstm_model = Sequential([
    LSTM(256, input_shape=(sequence_length, feature_dimension), return_sequences=True),
    Dropout(0.3),
    LSTM(128),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(num_composer_classes, activation='softmax')
])

# Compile the model with optimizer, loss, and metric
composer_lstm_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Display model summary
composer_lstm_model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                   │ (None, 168, 256)       │       350,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 168, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 128)            │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 555,844 (2.12 MB)

 Trainable params: 555,844 (2.12 MB)

 Non-trainable params: 0 (0.00 B)

In [67]:
# Train the model for 10 epochs with batch size 32
training_history = composer_lstm_model.fit(
    x_train_all, y_train_all_encoded,
    epochs=10,
    batch_size=32,
    validation_data=(x_test_all, y_test_all_encoded)
)

Epoch 1/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.4494 - loss: 1.2864 - val_accuracy: 0.5000 - val_loss: 0.9281
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.4527 - loss: 0.9044 - val_accuracy: 0.5000 - val_loss: 0.8423
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.4688 - loss: 0.8702 - val_accuracy: 0.5000 - val_loss: 0.8322
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.5437 - loss: 0.7952 - val_accuracy: 0.5000 - val_loss: 0.8275
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.5056 - loss: 0.8380 - val_accuracy: 0.5000 - val_loss: 0.8280
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 14s 999ms/step - accuracy: 0.4974 - loss: 0.8461 - val_accuracy: 0.5000 - val_loss: 0.8303
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.4709 - loss: 0.8542 - val_accuracy: 0.5000 - val_loss: 0.8271
Epoch 8/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 20s 926ms/step - accuracy: 0.4982 - loss: 0.8398 - val_accuracy: 0.5000 - val

In [68]:
# Evaluate the model on test set
test_loss, test_accuracy = composer_lstm_model.evaluate(x_test_all, y_test_all_encoded)
print(f"Test accuracy: {test_accuracy:.4f}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 481ms/step - accuracy: 0.3958 - loss: 0.8155
Test accuracy: 0.5000


In [69]:
# Perform predictions on test set
test_predictions = composer_lstm_model.predict(x_test_all)
predicted_labels = np.argmax(test_predictions, axis=1)

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 458ms/step


In [70]:
print("\n" + "="*40)
print("        Expected Labels")
print("="*40)
print(y_test_all_encoded)
print("\n")

print("="*40)
print("        Predicted Labels")
print("="*40)
print(predicted_labels)
print("\n")

# Print classification report for in-depth metrics
classification_rep = classification_report(y_test_all_encoded, predicted_labels)
print("="*40)
print("      CLASSIFICATION REPORT")
print("="*40)
print(classification_rep)


        Expected Labels
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3]


        Predicted Labels
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3]


      CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        28
           1       1.00      1.00      1.00        28
           2       0.00      0.00      0.00        28
           3       0.33      1.00      0.50        28

    accuracy                           0.50       112
   macro avg       0.33      0.50      0.38       112
weighted avg       0.33      0.50      0.38       112

